# 3.1 텐서와 Autograd

## 3.1.1 텐서 다루기 기본:  차원(Rank)과 Shape

In [10]:
import numpy as np
import torch
from tensorflow.keras import backend

x = torch.tensor([[1,2,3], [4,5,6], [7,8,9]])
print(x)
print("Size:", x.size())
print("Shape:", x.shape)
print("랭크(차원):", x.ndimension())

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
Size: torch.Size([3, 3])
Shape: torch.Size([3, 3])
랭크(차원): 2


In [3]:
x = torch.tensor([[1,2,3]])
print(x)
print("Size:", x.size())
print("Shape:", x.shape)
print("랭크(차원):", x.ndimension())

tensor([[1, 2, 3]])
Size: torch.Size([1, 3])
Shape: torch.Size([1, 3])
랭크(차원): 2


In [4]:
x = torch.tensor([[1],[2],[3]])
print(x)
print("Size:", x.size())
print("Shape:", x.shape)
print("랭크(차원):", x.ndimension())

tensor([[1],
        [2],
        [3]])
Size: torch.Size([3, 1])
Shape: torch.Size([3, 1])
랭크(차원): 2


# unsqueeze

In [25]:
type(np.newaxis)

NoneType

In [21]:
base_array = [[1,2,3], [4,5,6], [7,8,9]]

numpy_array = np.array(base_array)
torch_tensor = torch.tensor(base_array)
keras_tensor = backend.constant(base_array)

print(numpy_array.shape, torch_tensor.shape, keras_tensor.shape)
print(np.expand_dims(numpy_array, axis=0).shape, 
      torch_tensor.unsqueeze(dim=0).shape,
     backend.expand_dims(keras_tensor, axis=0).shape)
print(numpy_array[None].shape, torch_tensor[None].shape, keras_tensor[None].shape)
print(numpy_array[:, None].shape, torch_tensor[:, None].shape, keras_tensor[:, None].shape)

(3, 3) torch.Size([3, 3]) (3, 3)
(1, 3, 3) torch.Size([1, 3, 3]) (1, 3, 3)
(1, 3, 3) torch.Size([1, 3, 3]) (1, 3, 3)
(3, 1, 3) torch.Size([3, 1, 3]) (3, 1, 3)


In [31]:
base_array = np.ones((1, 3, 3, 1))

numpy_array = np.array(base_array)
torch_tensor = torch.tensor(base_array)
keras_tensor = backend.constant(base_array)
print(numpy_array.shape, torch_tensor.shape, keras_tensor.shape)
print(np.squeeze(numpy_array).shape, 
      torch_tensor.squeeze().shape,
     backend.squeeze(keras_tensor, axis=0).shape)
print(np.squeeze(numpy_array, axis=0).shape, 
      torch_tensor.squeeze(dim=0).shape,
     backend.squeeze(keras_tensor, axis=0).shape)

(1, 3, 3, 1) torch.Size([1, 3, 3, 1]) (1, 3, 3, 1)
(3, 3) torch.Size([3, 3]) (3, 3, 1)
(3, 3, 1) torch.Size([3, 3, 1]) (3, 3, 1)


# X 의 shape (512,512) => x.view(-1,256,256) => x.shape = (4,256,256)

In [45]:
base_array = np.ones((32, 3, 256, 256))
keras_array = np.ones((32, 256, 256, 3))

numpy_array = np.array(base_array)
torch_tensor = torch.tensor(base_array)
keras_tensor = backend.constant(keras_array)


reshape_keras_tensor = backend.permute_dimensions(keras_tensor, (0, 3, 1, 2))
reshape_keras_tensor = backend.reshape(keras_tensor, (96, 1, 256, 256))

In [46]:
print(numpy_array.reshape(-1, 1, 256, 256).shape)
print(torch_tensor.reshape(-1, 1, 256, 256).shape)
print(reshape_keras_tensor.shape)

(96, 1, 256, 256)
torch.Size([96, 1, 256, 256])
(96, 1, 256, 256)


# tile

In [71]:
base_array = [[1, 2, 3]]

tile_coef = [2, 4]

numpy_array = np.array(base_array)
torch_tensor = torch.tensor(base_array)
keras_tensor = backend.constant(keras_array)

print(np.tile(numpy_array, [5, 3]).shape)
print(torch.tile(torch_tensor, dims=[5, 3]).shape)

(5, 9)
torch.Size([5, 9])


In [20]:
import torch
from torch.nn import Softmax
# 랭크의 형태 바꾸기
x = torch.randn(2,10)
print("Size:", x.size())
x = Softmax(dim=1)(x) # (?,1)
print(x)
print("Shape:", x.shape)

Size: torch.Size([2, 10])
tensor([[0.0806, 0.1110, 0.0649, 0.1206, 0.0430, 0.0909, 0.1475, 0.1461, 0.1366,
         0.0587],
        [0.0733, 0.2067, 0.0088, 0.1746, 0.0784, 0.0467, 0.0785, 0.0747, 0.2232,
         0.0351]])
Shape: torch.Size([2, 10])


# Torch Layer 내부 이해하기

In [77]:
from torch import nn

test_layer = nn.Linear(32, 64, bias=True)

print(test_layer.weight.shape)
print(test_layer.bias.shape)

torch.Size([64, 32])
torch.Size([64])


In [79]:
test_layer = nn.Conv2d(32, 64, 3, bias=True)

print(test_layer.weight.shape)
print(test_layer.bias.shape)

torch.Size([64, 32, 3, 3])
torch.Size([64])


# 간단한 모델을 다양한 방법으로 만들기

# Simple

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(3, 32)
        self.layer_2 = nn.Linear(32, 64)
    
    def forward(self, x):
        x = self.layer_1(x)
        x = self.layer_2(x)
        return x

# Sequential

In [101]:
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_input = nn.Linear(3, 32)
        self.layer = nn.Sequential(
            nn.Linear(32, 64),
            nn.Linear(64, 128)
        )
    
    def forward(self, x):
        # x.shape: [B, 3]
        x = self.layer_input(x)
        # x.shape: [B, 32]
        x = self.layer(x)
        # x.shape: [B, 32] => [B, 64] => [B, 128]
        return x
    
class SimpleModel(nn.Module):
    def __init__(self, depth=2):
        super().__init__()
        self.layer_input = nn.Linear(3, 32)
        layer_list = []
        for idx in range(1, depth + 1):
            layer = nn.Linear(32 * idx, 32 * (idx + 1))
            layer_list.append(layer)
        self.layer = nn.Sequential(*layer_list)
    
    def forward(self, x):
        # x.shape: [B, 3]
        x = self.layer_input(x)
        # x.shape: [B, 32]
        x = self.layer(x)
        # x.shape: [B, 32] => [B, 64] => [B, 128]
        return x

# ModuleList

In [120]:
print(isinstance("a", str))
print(isinstance(test_layer, nn.Module))

True
True


In [118]:
class SimpleModel(nn.Module):
    def __init__(self, depth=2):
        super().__init__()
        self.layer_input = nn.Linear(3, 32)
        layer_list = [nn.Linear(32 * idx, 32 * (idx + 1))
                           for idx in range(1, depth + 1)]
        self.layer_list = nn.ModuleList(layer_list)
    def forward(self, x):
        # x.shape: [B, 3]
        x = self.layer_input(x)
        # x.shape: [B, 32]
        for layer in self.layer_list:
            x = layer(x)
        # x.shape: [B, 32] => [B, 64] => [B, 128]
        return x

# ModuleDict

In [129]:
{f"{idx}": idx ** 2 for idx in range(10)}

{'0': 0,
 '1': 1,
 '2': 4,
 '3': 9,
 '4': 16,
 '5': 25,
 '6': 36,
 '7': 49,
 '8': 64,
 '9': 81}

In [131]:
class SimpleModel(nn.Module):
    def __init__(self, depth=2):
        super().__init__()
        self.layer_input = nn.Linear(3, 32)
        layer_dict = {f"layer_{idx}":nn.Linear(32 * idx, 32 * (idx + 1))
                       for idx in range(1, depth + 1)}
        self.layer_dict = nn.ModuleDict(layer_dict)
        
    def forward(self, x):
        # x.shape: [B, 3]
        x = self.layer_input(x)
        # x.shape: [B, 32]
        for layer_name, layer in self.layer_dict.items():
            x = layer(x)
        # x.shape: [B, 32] => [B, 64] => [B, 128]
        return x

In [133]:
base_array = np.ones((3, 3))
# torch_tensor = torch.tensor(base_array, dtype=torch.float32)
torch_tensor = torch.tensor(base_array).float().cuda()

model = SimpleModel(depth=5).cuda()
print(model.layer_dict)
print(model(torch_tensor).shape)

ModuleDict(
  (layer_1): Linear(in_features=32, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=96, bias=True)
  (layer_3): Linear(in_features=96, out_features=128, bias=True)
  (layer_4): Linear(in_features=128, out_features=160, bias=True)
  (layer_5): Linear(in_features=160, out_features=192, bias=True)
)
torch.Size([3, 192])


In [ ]:
uint8
float32 => Float
float64 => Double

In [83]:
torch_tensor.dtype

torch.float64